<a href="https://colab.research.google.com/github/chldmstj/CHAI/blob/master/KerasRetina_Raccoon_Training_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [ ]:
!git clone https://github.com/chulminkw/keras-retinanet-tf115.git keras-retinanet

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 149, done.
remote: Total 149 (delta 0), reused 0 (delta 0), pack-reused 149
Receiving objects: 100% (149/149), 2.12 MiB | 2.06 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [ ]:
%cd /content/keras-retinanet/

/content/keras-retinanet


In [ ]:
!pwd

/content/keras-retinanet


In [ ]:
%cd /content/keras-retinanet/
!pip install . --user
!python setup.py build_ext --inplace

/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=145831 sha256=1a0118cfe484304fcbf677803470737ac4e5b0a7d1c9b4e9122ac84d4c123cff
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20487 sha256=1bb804c61a6ab2392528f860926d8d967d10fea775864c91da48d8f3133142cd
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-retinanet keras-resnet
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'l

### Raccoon 데이터 세트를 학습하고 학습된 모델을 이용하여 이미지와 비디오에 Object Detection과 성능 평가. 

In [ ]:
!rm -rf /content/raccoon_dataset

In [ ]:
!git clone "https://github.com/experiencor/raccoon_dataset.git" "/content/raccoon_dataset"

Cloning into '/content/raccoon_dataset'...
remote: Enumerating objects: 646, done.
remote: Total 646 (delta 0), reused 0 (delta 0), pack-reused 646
Receiving objects: 100% (646/646), 48.00 MiB | 19.80 MiB/s, done.
Resolving deltas: 100% (412/412), done.


#### Raccoon 데이터 세트의 image와 annotation 디렉토리 설정

In [ ]:
import os

ANNO_DIR = os.path.join('/content', 'raccoon_dataset','annotations')
IMAGE_DIR = os.path.join('/content', 'raccoon_dataset','images')

#### keras-retina 패키지의 사용 가능한 데이터 포맷
* keras-retina 패키지는 VOC, COCO, Open Image, 그리고 csv 형태의 데이터포맷을 모두 사용 가능 
* 하지만 VOC,COCO, OpenImage 모두 경연대회에서 사용된 디렉토리 구조가 필요함.   
* Raccoon Dataset을 csv 형태의 데이터 포맷을 활용하여 데이터 입력 적용
* csv 형태의 annotation과 class mapping format 필요.  
  * annotation은 아래와 같이 표현 가능  
    * /data/imgs/img_001.jpg,837,346,981,456,cow
  * 하나의 이미지 파일에서 두개 이상의 오브젝트가 있다면 두개 이상의 라인으로 정보 표시.   
    * /data/imgs/img_002.jpg,215,312,279,391,cat  
    * /data/imgs/img_002.jpg,22,5,89,84,bird  

#### 학습과 검증 데이터 세트를 위한 별도의 annotation파일 생성. 
* keras-retina패키지는 validation annotation파일을 이용하여 학습 시 evaluation 수행 가능
* 아래는 80%의 xml파일을 train csv로, 나머지 20% xml파일은 valid csv 로 생성

In [ ]:
import numpy as np

In [ ]:
xml_files = [xml_file for xml_file in os.listdir(ANNO_DIR)]

In [ ]:
len(xml_files)

200

In [ ]:
xml_files = np.array(xml_files)

In [ ]:
xml_files.shape

(200,)

In [ ]:
total_indexes = np.arange(0, total_cnt)

In [ ]:
total_indexes

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [ ]:
valid_indexes = np.random.choice(total_indexes, valid_cnt, replace=False)

In [ ]:
valid_indexes

array([174,  73,  47,  44, 164, 122, 134,  46,  90,  64, 106, 155, 179,
        63,   6, 196, 157, 184,   3,  65, 105, 133, 108, 186,  99,  37,
       114, 193,  27, 181,  76, 151,  83,  69,  42,  31,  50, 136, 197,
        96])

In [ ]:
train_indexes = total_indexes[~np.isin(total_indexes, valid_indexes)]

In [ ]:
train_indexes

array([  0,   1,   2,   4,   5,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  28,
        29,  30,  32,  33,  34,  35,  36,  38,  39,  40,  41,  43,  45,
        48,  49,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  66,  67,  68,  70,  71,  72,  74,  75,  77,  78,  79,  80,
        81,  82,  84,  85,  86,  87,  88,  89,  91,  92,  93,  94,  95,
        97,  98, 100, 101, 102, 103, 104, 107, 109, 110, 111, 112, 113,
       115, 116, 117, 118, 119, 120, 121, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 135, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 152, 153, 154, 156, 158, 159, 160,
       161, 162, 163, 165, 166, 167, 168, 169, 170, 171, 172, 173, 175,
       176, 177, 178, 180, 182, 183, 185, 187, 188, 189, 190, 191, 192,
       194, 195, 198, 199])

In [ ]:
def get_train_valid_indexes(anno_path, valid_size ):
    np.random.seed(0)
    
    xml_files = [xml_file for xml_file in os.listdir(anno_path)]
    xml_files = np.array(xml_files)
    total_cnt = xml_files.shape[0]
    valid_cnt = int(total_cnt * valid_size)
    
    total_indexes = np.arange(0, total_cnt)
    valid_indexes = np.random.choice(total_indexes, valid_cnt, replace=False)
    train_indexes = total_indexes[~np.isin(total_indexes, valid_indexes)]
    
    return train_indexes, valid_indexes

In [ ]:
train_indexes, valid_indexes = get_train_valid_indexes(ANNO_DIR, 0.2)
train_indexes.shape, valid_indexes.shape

((160,), (40,))

#### csv annotation 데이터 파일을 만들기 위한 함수 생성. 
* 인자로 annotation 디렉토리명, csv형태로 만들어질 파일명을 주면 생성파일명으로 csv 형태의 annotation 데이터 파일 생성.  

In [ ]:
import glob
import xml.etree.ElementTree as ET

In [ ]:
xml_list = np.array([xml_file for xml_file in glob.glob(ANNO_DIR + '/*.xml')])

In [ ]:
xml_list[0]

'/content/raccoon_dataset/annotations/raccoon-19.xml'

In [ ]:
tree = ET.parse(xml_list[0])
type(tree)

xml.etree.ElementTree.ElementTree

In [ ]:
root = tree.getroot()
type(root)

xml.etree.ElementTree.Element

In [ ]:
root.find('filename').text

'raccoon-19.jpg'

In [ ]:
for obj in root.findall('object'):
  print(type(obj))
  xmlbox = obj.find('bndbox')
  x1 = int(xmlbox.find('xmin').text)
  y1 = int(xmlbox.find('ymin').text)
  x2 = int(xmlbox.find('xmax').text)
  y2 = int(xmlbox.find('ymax').text)
  value_str = ('{0},{1},{2},{3}').format(x1, y1, x2, y2)
  print(value_str)

<class 'xml.etree.ElementTree.Element'>
87,8,182,89


In [ ]:
def xml_to_csv_sampling(path, output_filename, sample_index):
    xml_list = np.array([xml_file for xml_file in glob.glob(path + '/*.xml')])
    xml_list = xml_list[sample_index]

    # xml 확장자를 가진 모든 파일의 절대 경로로 xml_file할당. 
    with open(output_filename, "w") as train_csv_file:
        for xml_file in xml_list:
            # xml 파일을 parsing하여 XML Element형태의 Element Tree를 생성하여 object 정보를 추출. 
            tree = ET.parse(xml_file)
            root = tree.getroot()
            # 파일내에 있는 모든 object Element를 찾음. 
            full_image_name = os.path.join(IMAGE_DIR, root.find('filename').text)
            value_str_list = ' '
            for obj in root.findall('object'):
                xmlbox = obj.find('bndbox')
                x1 = int(xmlbox.find('xmin').text)
                y1 = int(xmlbox.find('ymin').text)
                x2 = int(xmlbox.find('xmax').text)
                y2 = int(xmlbox.find('ymax').text)
                # 단 하나의 
                class_name='raccoon'
                value_str = ('{0},{1},{2},{3},{4},{5}').format(full_image_name,x1, y1, x2, y2, class_name)
                # object별 정보를 tuple형태로 object_list에 저장. 
                train_csv_file.write(value_str+'\n')
        # xml file 찾는 for loop 종료 

In [ ]:
train_indexes, valid_indexes = get_train_valid_indexes(ANNO_DIR, 0.2)
xml_to_csv_sampling(ANNO_DIR, os.path.join(ANNO_DIR,'raccoon_anno_retina_train.csv'), train_indexes)
xml_to_csv_sampling(ANNO_DIR, os.path.join(ANNO_DIR,'raccoon_anno_retina_valid.csv'), valid_indexes)

In [ ]:
with open('/content/raccoon_dataset/annotations/raccoon_class.txt', "w") as f:
    f.write("raccoon,0")

#### keras_retinanet/bin/train.py를 이용하여 학습 수행. 
* keras-retinanet 패키지는 학습시간이 비교적 오래 필요. 
* 특히 batch-size 가 크게 설정하기 어려움. 2이상 설정 시 메모리를 과다 사용으로 segmentation fault 오류 발생.
* Shell에서 export TF_CUDNN_USE_AUTOTUNE=0  설정하면 batch-size를 2로 늘릴 수 있으나 큰 학습 시간 단축은 기대하기 어려움. 
* Raccoon 데이터 세트는 steps=200, epochs=20 정도면 충분한 학습이 가능. 
* 학습 시 epoch가 완료될 때마다 snapshots 디렉토리에 모델들을 계속 생성하여 저장. 
* train.py는 많은 환경 변수를 명령 인자로 입력해야함. 명령 인자로 입력하지 않을 경우 Default 환경 변수값으로 입력됨. Default 환경 변수값에 대한 이해 필요. 

In [ ]:
!chmod 777 ./keras_retinanet/bin/train.py

In [ ]:
# 아래는 shell에서 수행해야 합니다. 
! ./keras_retinanet/bin/train.py --epochs=20 --steps=200 \
  csv /content/raccoon_dataset/annotations/raccoon_anno_retina_train.csv \
      /content/raccoon_dataset/annotations/raccoon_class.txt \
      --val-annotations=/content/raccoon_dataset/annotations/raccoon_anno_retina_valid.csv \


Using TensorFlow backend.


2020-10-21 11:47:06.204775: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-10-21 11:47:06.258887: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000134999 Hz
2020-10-21 11:47:06.259192: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x17daa00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-10-21 11:47:06.259232: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-10-21 11:47:06.264211: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-21 11:47:06.429723: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-1

#### train.py의 여러 모듈을 직접 import하여 학습 수행. 
* train.py의 여러 모듈을 직접 import하여 customization으로 학습을 수행하는 것이 더 직관적이고 빠른 학습 시간 보장.
* keras-retinanet으로 학습 시 어떻게 내부 모듈이 동작하는지 더 명확히 알 수 있음. 
* 환경 파라미터를 훨씬 편하게 조정 가능

In [ ]:
import cv2
from os import listdir, walk
import math
import tensorflow as tf
from os.path import join
from keras_retinanet.bin.train import create_generators,create_models,create_callbacks
from keras_retinanet.models import backbone,load_model,convert_model
from keras_retinanet.utils.config import read_config_file,parse_anchor_parameters
from keras_retinanet.utils.visualization import draw_boxes

#from imgaug import augmenters as iaa

tf.set_random_seed(31) # SEEDS MAKE RESULTS MORE REPRODUCABLE
np.random.seed(17)

Using TensorFlow backend.


#### 환경 파라미터 설정. 

In [ ]:
b = backbone('resnet50')
files = os.listdir(ANNO_DIR)
train_file_cnt = train_indexes.shape[0]

class args:
    batch_size = 4
    config = None
    random_transform = True # Image augmentation
    annotations = os.path.join(ANNO_DIR, 'raccoon_anno_retina_train.csv')
    val_annotations = os.path.join(ANNO_DIR, 'raccoon_anno_retina_valid.csv')
    classes = os.path.join(ANNO_DIR, 'raccoon_class.txt')
    image_min_side = 800
    image_max_side = 1333
    no_resize=None
    dataset_type = 'csv'
    tensorboard_dir = ''
    evaluation = True
    snapshots = True
    snapshot_path = './keras-retinanet/snapshots'
    backbone = 'resnet50'
    epochs = 20
    steps = train_file_cnt//(batch_size)
    weighted_average = True

ModuleNotFoundError: ignored

#### 학습과 검증을 위한 generator 생성.

In [ ]:
train_gen,valid_gen = create_generators(args,b.preprocess_image)

#### backend CNN과 기타 환경 설정하여 기본 모델 생성

In [ ]:
model, training_model, prediction_model = create_models(
            backbone_retinanet=b.retinanet,
            num_classes=train_gen.num_classes(),
            weights=None,
            multi_gpu=False,
            freeze_backbone=True,
            lr=1e-3,
            config=args.config
        )

#### Checkpoint, ReduceLROnPlateur와 같은 callback 기능 생성. 

In [ ]:
callbacks = create_callbacks(
    model,
    training_model,
    prediction_model,
    valid_gen,
    args,
)

#### 학습 모델에 coco로 pretrained된 weight를 최초 weight로 설정

In [ ]:
training_model.load_weights('./keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5',skip_mismatch=True,by_name=True)

In [ ]:
training_model.fit_generator(generator=train_gen,
        steps_per_epoch=args.steps,
        epochs=args.epochs,
        verbose=1,
        validation_data=valid_gen,                     
        callbacks=callbacks)

### 학습 모델 기반 Object Detection 및 Detection 성능 평가(Evaluation)

#### 학습 모델을 Inference 모델로 변환
*  keras_retinanet/bin/convert_model.py를 이용하여 snapshots 디렉토리에 가장 마지막에 만들어진 학습 모델(가장 손실율이 적은)을 infererence용 모델로 변환

In [ ]:
import sys
import os
ROOT_DIR = os.path.abspath(".")
sys.path.append(ROOT_DIR)

#!./keras-retinanet/keras_retinanet/bin/convert_model.py ~/DLCV/Detection/retina/keras-retinanet/snapshots/resnet50_csv_20.h5 \
#~/DLCV/Detection/retina/keras-retinanet/snapshots/raccoon_inference.h5


#### 변환된 inference용 모델인 raccoon_inference.h5 파일을 로드하여 이미지 Detection 수행

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time


from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
setup_gpu(gpu)

In [ ]:
import os
import sys

ROOT_DIR = os.path.abspath(".")
sys.path.append(ROOT_DIR)

model_path = os.path.join(ROOT_DIR, 'keras-retinanet/snapshots/raccoon_inference.h5')

print(model_path)
# load retinanet model
raccoon_retina_model = models.load_model(model_path, backbone_name='resnet50')

#### 이미지 detect를 위한 함수 생성. 
* inference를 수행하기 전에 이미지 scaling 및 크기를 재 조정할 수 있도록 preprocess_image()와 resize_image() 제공. 
* keras-retinanet은 이미지에 bounding box를 편리하게 그릴 수 있는 API제공. draw_box(), draw_caption(), label_color() 제공

In [ ]:
import cv2
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

labels_to_names_seq = {0:'Raccoon'}

def get_detected_image_retina(model, img_array, use_copied_array, is_print=True):
    
    # copy to draw on
    draw_img = None
    if use_copied_array:
        draw_img = img_array.copy()
    else:
        draw_img = img_array
    
    img_array = preprocess_image(img_array)
    img_array, scale = resize_image(img_array)
    
    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(img_array, axis=0))
    if is_print:
        print("object detection 처리 시간: ", round(time.time() - start,5))
    
    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw_img, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names_seq[label], score)
        draw_caption(draw_img, b, caption)
    
    if is_print:
        print("이미지 processing 시간: ", round(time.time() - start,5))
    
    return draw_img

In [ ]:
import os
from pathlib import Path

HOME_DIR = str(Path.home())

ANNO_DIR = os.path.join(HOME_DIR, 'DLCV/data/raccoon/annotations')
IMAGE_DIR = os.path.join(HOME_DIR, 'DLCV/data/raccoon/images')

img_array  = cv2.imread(os.path.join(IMAGE_DIR, 'raccoon-22.jpg'))
draw_img_array = img_array.copy()
draw_img_array = cv2.cvtColor(draw_img_array, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(draw_img_array)
plt.show()

detected_image = get_detected_image_retina(raccoon_retina_model, img_array, use_copied_array=True, is_print=True)
img_rgb = cv2.cvtColor(detected_image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(img_rgb)
plt.show()

In [ ]:
import numpy as np
np.random.seed(0)

# 모든 이미지 파일중에서 임의의 16개 파일만 설정. 
all_image_files = glob.glob(IMAGE_DIR + '/*.jpg')
all_image_files = np.array(all_image_files)
file_cnt = all_image_files.shape[0]
show_cnt = 16

show_indexes = np.random.choice(file_cnt, show_cnt)
show_files = all_image_files[show_indexes]
print(show_files)
fig, axs = plt.subplots(figsize=(24,24) , ncols=4 , nrows=4)

for i , filename in enumerate(show_files):
    print(filename)
    row = int(i/4)
    col = i%4
    img_array = cv2.imread(os.path.join(IMAGE_DIR, filename))
    detected_image = get_detected_image_retina(raccoon_retina_model,img_array, use_copied_array=True, is_print=True)
    img_rgb = cv2.cvtColor(detected_image, cv2.COLOR_BGR2RGB)
    axs[row][col].imshow(img_rgb)

#### video에 object detection을 수행
* get_detected_image()와 유사한 함수를 생성. 인자로 image array와 retina 모델을 입력, 개별 frame별로 object Detection 수행. 

In [ ]:
def detect_video_retina(model, input_path, output_path=""):
    
    start = time.time()
    cap = cv2.VideoCapture(input_path)
    
    codec = cv2.VideoWriter_fourcc(*'XVID')
    vid_fps = cap.get(cv2.CAP_PROP_FPS)
    vid_size= (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size)
    
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('총 Frame 갯수:', frame_cnt)
    
    while True:
        hasFrame, image_frame = cap.read()
        if not hasFrame:
            print('프레임이 없거나 종료 되었습니다.')
            break

        detected_image = get_detected_image_retina(model,image_frame, use_copied_array=False, is_print=True)
        vid_writer.write(detected_image)
    
    vid_writer.release()
    cap.release()
    print('### Video Detect 총 수행시간:', round(time.time()-start, 5))

In [ ]:
detect_video_retina(raccoon_retina_model, input_path='../../data/video/jack_and_raccoon.mp4', output_path='../../data/output/jack_retina.avi')

In [ ]:
!gsutil cp ~/DLCV/data/output/jack_retina.avi gs://my_bucket_dlcv/data/output/jack_retina.avi

#### Raccoon 데이터 세트 학습 모델의 Object Detection 성능 평가

In [ ]:
from keras_retinanet.bin.evaluate import create_generator as eval_create_generator

In [ ]:
import os
from pathlib import Path

HOME_DIR = str(Path.home())
ANNO_DIR = os.path.join(HOME_DIR, 'DLCV/data/raccoon/annotations')

class args:
    dataset_type='csv'
    score_threshold=0.05
    iou_threshold=0.5
    max_detections=100
    image_min_side=800
    image_max_side=1333
    config=None
    annotations=os.path.join(ANNO_DIR, 'raccoon_anno_retina_valid.csv')
    classes=os.path.join(ANNO_DIR, 'raccoon_class.txt')

In [ ]:
generator = eval_create_generator(args)

In [ ]:
from keras_retinanet.utils.eval import evaluate

average_precisions, inference_time = evaluate(
            generator,
            raccoon_retina_model,
            iou_threshold=args.iou_threshold,
            score_threshold=args.score_threshold,
            max_detections=args.max_detections,
            save_path=None
        )

In [ ]:
# print evaluation
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations),
          generator.label_to_name(label), 'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)

if sum(total_instances) == 0:
    print('No test instances found.')

print('Inference time for {:.0f} images: {:.4f}'.format(generator.size(), inference_time))

print('mAP using the weighted average of precisions among classes: {:.4f}'.format(sum([a * b for a, b in zip(total_instances, precisions)]) / sum(total_instances)))
print('mAP: {:.4f}'.format(sum(precisions) / sum(x > 0 for x in total_instances)))